In [1]:
addprocs()

@everywhere begin
  using StochasticDiffEq, DiffEqProblemLibrary, ParallelDataTransfer
  srand(99 + myid())
  prob = oval2ModelExample(largeFluctuations=true,useBigs=false)
  prob.tspan = (0.0,1.0)
  println("Solve once to compile.")
  sol = solve(prob,EM(),dt=1/2^(18),adaptive=false,save_everystep=false)
  sol = solve(prob,RKMil(),dt=1/2^(18),adaptive=false,save_everystep=false)
  sol = solve(prob,SRIW1(),dt=1/2^(18),adaptive=false,save_everystep=false)
  sol = solve(prob,SRI(),dt=1/2^(18),adaptive=false,save_everystep=false)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt1()),dt=1/2^(18),adaptive=false,save_everystep=false)
  Int(sol.u[1]!=NaN)
  println("Compilation complete.")
  js = 16:21
  dts = 1./2.^(js)
  fails = Array{Int}(length(dts),3)
  times = Array{Float64}(length(dts),3)
  numRuns = 10000
end
println("Setup Complete")

Solve once to compile.
Compilation complete.
	From worker 13:	Solve once to compile.
	From worker 16:	Solve once to compile.
	From worker 16:	Compilation complete.
	From worker 7:	Solve once to compile.
	From worker 4:	Solve once to compile.
	From worker 4:	Compilation complete.
	From worker 11:	Solve once to compile.
	From worker 11:	Compilation complete.
	From worker 5:	Solve once to compile.
	From worker 2:	Solve once to compile.
	From worker 2:	Compilation complete.
	From worker 14:	Solve once to compile.
	From worker 8:	Solve once to compile.
	From worker 8:	Compilation complete.
	From worker 17:	Solve once to compile.
	From worker 17:	Compilation complete.
	From worker 6:	Solve once to compile.
	From worker 6:	Compilation complete.
	From worker 9:	Solve once to compile.
	From worker 3:	Solve once to compile.
	From worker 3:	Compilation complete.
	From worker 12:	Solve once to compile.
	From worker 10:	Solve once to compile.
	From worker 15:	Solve once to compile.
	From worker 15:

In [19]:
## Timing Runs

@everywhere function runAdaptiveSRIW1(i)
  sol = solve(prob,SRIW1(),abstol=2.0^(-12),reltol=2.0^(-6),maxIters=Int(1e11),qmax=1.125,save_everystep=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIW1,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 12.99248294


In [72]:
## Timing Runs

@everywhere function runAdaptiveSRI(i)
  sol = solve(prob,SRI(error_terms=2),abstol=2.0^(-12),reltol=2.0^(-7),maxIters=Int(1e11),qmax=1.125,save_everystep=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRI,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 32.503944551


In [80]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt1(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt1()),abstol=2.0^(-7),reltol=2.0^(-3),maxIters=Int(1e11),qmax=1.125,save_everystep=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt1,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 4.459500891


In [60]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt2(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt2()),abstol=2.0^(-9),reltol=2.0^(-6),maxIters=Int(1e11),qmax=1.125,save_everystep=false)
    Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt2,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 8.678555793


In [52]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt3(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt3()),abstol=2.0^(-7),reltol=2.0^(-5),maxIters=Int(1e10),qmax=1.125,save_everystep=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt3,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 7.326458853


In [44]:
## Timing Runs

@everywhere function runAdaptiveSRIOpt4(i)
  sol = solve(prob,SRI(tableau=StochasticDiffEq.constructSRIOpt4()),abstol=2.0^(-8),reltol=2.0^(-4),maxIters=Int(1e10),qmax=1.125,save_everystep=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
@everywhere srand(99 + myid())
adaptiveTime = @elapsed numFails = sum(pmap(runAdaptiveSRIOpt4,1:numRuns))
println("The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")

The number of Adaptive Fails is 0. Elapsed time was 7.068097671


In [25]:
@everywhere function runEM(i,j)
  sol =solve(prob,EM(),dt=dts[j],maxIters=Int(1e11),save_everystep=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in eachindex(js)
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t1 = @elapsed numFails = sum(pmap((i)->runEM(i,j),1:numRuns))
  println("The number of Euler-Maruyama Fails is $numFails. Elapsed time was $t1")
  fails[j,1] = numFails
  times[j,1] = t1
end

j = 1
The number of Euler-Maruyama Fails is 130. Elapsed time was 22.485178841
j = 2
The number of Euler-Maruyama Fails is 35. Elapsed time was 42.220078027
j = 3
The number of Euler-Maruyama Fails is 6. Elapsed time was 84.478325558
j = 4
The number of Euler-Maruyama Fails is 0. Elapsed time was 169.96851466
j = 5
The number of Euler-Maruyama Fails is 0. Elapsed time was 342.955733199
j = 6
The number of Euler-Maruyama Fails is 0. Elapsed time was 686.696407877


In [26]:
@everywhere function runSRI(i,j)
  sol =solve(prob,SRIW1(),dt=dts[j],maxIters=Int(1e11),adaptive=false,save_everystep=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in 1:4
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t2 = @elapsed numFails = sum(pmap((i)->runSRI(i,j),1:numRuns))
  println("The number of Rossler-SRI Fails is $numFails. Elapsed time was $t2")
  fails[j,2] = numFails
  times[j,2] = t2
end

j = 1
The number of Rossler-SRI Fails is 90. Elapsed time was 51.434818926
j = 2
The number of Rossler-SRI Fails is 15. Elapsed time was 103.313398945
j = 3
The number of Rossler-SRI Fails is 2. Elapsed time was 209.047359334
j = 4
The number of Rossler-SRI Fails is 0. Elapsed time was 424.301395463


In [27]:
@everywhere function runMil(i,j)
  sol =solve(prob,RKMil(),dt=dts[j],maxIters=Int(1e11),save_everystep=false,verbose=false)
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end
for j in eachindex(js)
  println("j = $j")
  sendto(workers(), j=j)
  @everywhere srand(99 + myid())
  t3 = @elapsed numFails = sum(pmap((i)->runMil(i,j),1:numRuns))
  println("The number of RK-Milstein Fails is $numFails. Elapsed time was $t3")
  fails[j,3] = numFails
  times[j,3] = t3
end

j = 1
The number of RK-Milstein Fails is 136. Elapsed time was 24.146935932
j = 2
The number of RK-Milstein Fails is 36. Elapsed time was 48.227152874
j = 3
The number of RK-Milstein Fails is 6. Elapsed time was 90.608892005
j = 4
The number of RK-Milstein Fails is 0. Elapsed time was 182.597809791
j = 5
The number of RK-Milstein Fails is 0. Elapsed time was 367.733253661
j = 6
The number of RK-Milstein Fails is 0. Elapsed time was 749.600321343


In [ ]:
using Plots
lw = 3
p2 = plot(dts,times,xscale=:log2,yscale=:log2,guidefont=font(16),tickfont=font(14),yguide="Elapsed Time (s)",xguide=L"Chosen $\Delta t$",top_margin=50px,linewidth=lw,lab=["Euler-Maruyama" "RK-Mil" "RosslerSRI"],legendfont=font(14))
plot!(dts,repmat([adaptiveTime],11),linewidth=lw,line=:dash,lab="ESRK+RSwM3",left_margin=75px)
scatter!([2.0^(-20);2.0^(-20);2.0^(-18)],[times[5,1];times[5,2];times[3,3]],markersize=20,c=:red,lab="")
plot(p2,size=(800,800))